In [10]:
from datetime import date

from pydataxm import pydataxm
import pandas as pd

In [11]:
API = pydataxm.ReadDB()

In [14]:
df = API.get_collections()

# with pd.option_context('display.max_rows', None, 'display.max_columns', None):
#     display(df)

In [ ]:
DispoCome = API.request_data(
    "DispoDeclarada",
    "Recurso",
    date(2023,6,1),
    date.today(),
)

In [ ]:
recursos = API.request_data(
    "ListadoRecursos",
    "Sistema",
    date.today(),
    date.today(),
)
company = API.request_data(
    "ListadoAgentes",
    "Sistema",
    date.today(),
    date.today(),
)

In [ ]:
indx_cols = ["Id", "Values_OperStartdate", "Values_State", "Date"]
recursos = recursos[[col for col in recursos.columns if col not in indx_cols]]
company = company[[col for col in company.columns if col not in indx_cols]]
agentes_company = recursos.merge(company, left_on="Values_CompanyCode", right_on="Values_Code", how="left", suffixes=("_resource", "_company"))
agentes_company.columns = [
    "resource_code", "resource_name", "gen_type", "dispatched", "resource_gen_type", "resource_company_code", "resource_source",
    "company_code", "resource_company_name", "company_activity"]
agentes_company.to_csv("data/agentes_company.csv", index=False)
agentes_company

In [ ]:
DispoCome_ = DispoCome.copy()
mapper = {f"Values_Hour{i+1:0>2}": i for i in range(24)}
indx = ["Id", "Values_code", "Date"]
DispoCome_.set_index(indx, inplace=True)
DispoCome_ = DispoCome_.rename(columns=mapper)
DispoCome_ = DispoCome_.stack().reset_index()
DispoCome_.columns = [*indx, "hora", "dispo"]
DispoCome_["datetime"] = pd.to_datetime(DispoCome_["Date"]) + pd.to_timedelta(DispoCome_["hora"], unit="h")
DispoCome_

In [ ]:
DispoCome_resource = DispoCome_.merge(
    agentes_company,
    left_on="Values_code",
    right_on="resource_code",
    how="left"
)
DispoCome_resource

In [21]:
DispoCome_resource.to_csv("data/dispo_declarada.csv", index=False)

# 3. Demanda Comercial

In [1]:
from pydataxm import pydataxm
from datetime import date
import pandas as pd

API = pydataxm.ReadDB()


In [ ]:
demaCome = API.request_data(
    "DemaCome",
    "Sistema",
    date(2023,1,1),
    date.today(),
)

In [27]:
demaCome_ = demaCome.copy()
demaCome_.set_index(["Id", "Values_code", "Date"], inplace=True)
demaCome_.columns = range(24)
demaCome_ = demaCome_.stack().reset_index()
demaCome_.columns = ["Id", "Values_code", "Date", "hora", "dema"]
demaCome_["datetime"] = pd.to_datetime(demaCome_["Date"]) + pd.to_timedelta(demaCome_["hora"], unit="h")
demaCome_.sort_values(["datetime"], inplace=True)
demaCome_ = demaCome_[["datetime", "dema"]]
demaCome_.to_csv("data/demaCome.csv", index=False)

In [ ]:
import plotly.express as px

px.line(
    demaCome_,
    x="datetime",
    y="dema",
    color="Values_code",
    title="Demanda de energía",
    labels={"dema": "Demanda [MW]", "datetime": "Fecha y hora", "Values_code": "Código"},
)

In [ ]:
df = API.get_collections()

df[df["MetricDescription"].str.lower().str.contains("reg")]

In [ ]:
API.request_data(
    "RespComerAGC",
    "Sistema",
    date(2024,2,15),
    date(2024,2,15),
)

In [ ]:
with pd.option_context('display.max_rows', None, 'display.max_columns', None):
    display(df)

In [ ]:
df_[df_["Values_code"] == "MOY1"]

# 4. AGC Asignado

In [ ]:
agc_df = pd.read_excel('data/AGC_Programado_(kWh)_2024.xlsx', parse_dates=["Fecha"])
agc_df = agc_df.fillna(0)
agc_df.set_index(['Fecha', "Recurso", "Código Agente"], inplace=True)
agc_df = agc_df.stack().reset_index()
agc_df.columns = ["fecha", "recurso", "codigo_agente", "hora", "agc"]
agc_df["datetime"] = pd.to_datetime(agc_df["fecha"]) + pd.to_timedelta(agc_df["hora"], unit='h')
agc_df["agc"] = agc_df["agc"].astype(float)
agc_df

In [16]:
agc_df.to_csv("data/agc_asignado.csv", index=False)